<a href="https://colab.research.google.com/github/vpdatacommons/VPDC_Notebooks/blob/main/notebooks/02_WildfireIgnitionProbability_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore Wildfire Ignition Probability data via Jupyter notebook

#### This notebook is relient on several free and open-source python packages, such as [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/), [Rasterio](https://rasterio.readthedocs.io/en/stable/), and [matplotlib](https://matplotlib.org/) that each contribute different tools in Python that empower users to do interactive mapping, geospatial analysis, and data visualization within a Jupyter notebook.

**<span style="color:red">NOTE: Many of these tools will need ample time to run depending on contributing factors such as the complexity of the data package, network speed, and server load.</span>**

Begin by installing necessary packages:

**Note:** If you already have python packages installed, comment out the code below by adding "#" in front of each line.
`# !pip install rasterio`

In [ ]:
#!pip install rasterio matplotlib numpy ipyleaflet ipywidgets

### Import the necessary packages:

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio.windows import from_bounds
from rasterio.windows import Window
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import ipyleaflet
from ipyleaflet import Map, TileLayer, Marker, Popup, WidgetControl
import ipywidgets as widgets

### Set geographic boundaries for your area of interest to create a subset of wildfire ignition probability dataset from VPDC:

In [ ]:
# ------ Configuration ------
cog_file_path = "https://vpdc-resources.s3.us-west-1.amazonaws.com/data-story/VibrantPlanet/IgnitionProbability/data/W_US/OPTIMIZED/W_Hum_Ign_COG.tif"  # Update with your S3 path
# ---------------------------

#  ------ Specify a bounding box (adjust for your data's CRS)  ------
# Example Bounding Box defined the geographic bounds of Northern California
geo_bounds = (-124.0, 38.0, -120.0, 42.0)  # (west_lon, south_lat, east_lon, north_lat)

# ------ Setup coordinate transformation from WGS 84 (EPSG:4326) to NAD83 / Conus Albers (EPSG:5070) ------
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:5070", always_xy=True)

# ------ Transform the bounds ------
min_x, min_y = transformer.transform(geo_bounds[0], geo_bounds[1])
max_x, max_y = transformer.transform(geo_bounds[2], geo_bounds[3])

### Load the data for visualization and analysis

In [ ]:
# ------ Data Loading ------
def load_subset(cog_file_path, desired_resolution=None):
    """Loads a spatial subset of the COG at the specified resolution.

    Args:
        cog_file_path (str): Path to the COG file.
        bounds (tuple): Bounding box in the format (left, bottom, right, top) in the CRS of your data.
        desired_resolution (float, optional): Target resolution in the units of your CRS.

    Returns:
        numpy.ndarray: The data subset at the desired resolution.
    """
    with rasterio.open(cog_file_path) as src:
        # Calculate the window to read
        window = from_bounds(min_x, min_y, max_x, max_y, src.transform)
        subset_data = src.read(1, window=window)

        print(f"Window: {window}")
        print(f"Subset shape: {subset_data.shape}")

        # Rescale if a different resolution is desired
        if desired_resolution:
            # Calculate scaling factors
            scale_x = desired_resolution / src.res[0]
            scale_y = desired_resolution / src.res[1]

            # Make sure scaling factors are sensible
            print(f"Scaling - X: {scale_x}, Y: {scale_y}")

            # Resample data to desired resolution
            subset_data = src.read(
                1,
                window=window,
                out_shape=(
                    int(subset_data.shape[0] * scale_y),
                    int(subset_data.shape[1] * scale_x)
                ),
                resampling=Resampling.bilinear
            )

    return subset_data

# Example of loading a 25-meter resolution subset
subset_data = load_subset(cog_file_path, desired_resolution=25)

### Basic visualization method

In [ ]:
# Visualize the subset of the data
def visualize_subset(subset_data, colormap, map_title='Human-Caused Wildfire Ignition Probability', legend_title= 'Probability'):
    """ Basic visualization using rasterio's show function. """
    fig, ax = plt.subplots(figsize=(10, 6))
    cmap = plt.get_cmap(colormap)
    # Use vmin and vmax to set the color scaling based on the data range
    img = ax.imshow(subset_data, cmap=cmap, vmin=np.nanmin(subset_data), vmax=np.nanmax(subset_data))
    fig.colorbar(img, ax=ax, orientation='horizontal', label=legend_title)
    ax.set_title(map_title)
    plt.show()

### Data exploration method

In [ ]:
# ------ Data Exploration ------
def calculate_statistics(data):
    """ Calculates basic statistics. """
    print("Min:", np.nanmin(data))
    print("Max:", np.nanmax(data))
    print("Mean:", np.nanmean(data))
    print("Standard Deviation:", np.nanstd(data))

### Data analysis workflow method

In [ ]:
# ------ Example Analysis Workflow ------
def fire_risk_thresholding(data, threshold=0.0001):
    """ Sample analysis: Identifies pixels exceeding a risk threshold."""
    risk_mask = data > threshold
    return risk_mask

# Example Usage

## Visualize the data

In [ ]:
# First, verify that values are returned
if subset_data.size > 0:
    # Visualize
    visualize_subset(subset_data, 'viridis')
    calculate_statistics(subset_data)
else:
    print("No data loaded in the subset. Please check bounding box coordinates and CRS.")

## Run basic statistics

In [ ]:
# Statistics
calculate_statistics(subset_data)

### Example of potential analysis workflow

In [ ]:
# Apply analysis workflow
risk_mask = fire_risk_thresholding(subset_data)
visualize_subset(risk_mask, 'RdYlBu', 'Areas exceeding a risk threshold', 'Threshold' )  # Visualize the results

### Example of interactive map with ipyleaflet

In [ ]:
# ------ Interactive Mapping ------
center = (43, -110)  # Approximate center of Western US
zoom = 5

map = ipyleaflet.Map(center=center, zoom=zoom)

# Add GeoTIFF as TileLayer
ignition_layer = ipyleaflet.TileLayer(
  url= cog_file_path,
  band=1,
  opacity=0.8,
  colormap=plt.cm.inferno,
  attribution="Wildfire Ignition Data"
)
map.add_layer(ignition_layer)

# Function to handle map clicks
def on_map_click(event, **kwargs):
    coords = event['latlng']
    # Simulate getting data for now
    print(f"Ignition Probability at ({coords[1]}, {coords[0]}): Sample Probability")
    # Show result in a popup
    popup = Popup(
        location=coords,
        child=widgets.HTML(value=f"<b>Latitude:</b> {coords[1]}<br><b>Longitude:</b> {coords[0]}"),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    map.add_layer(popup)

map.on_interaction(on_map_click)

# Example HTML widget for displaying instructions or information
info = widgets.HTML(value="Click on the map to see ignition probability.", placeholder='', description='')
widget_control = WidgetControl(widget=info, position='topright')
map.add_control(widget_control)

# Display the map
map